In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = pd.read_excel("/content/drive/MyDrive/hospital.xlsx")

In [ ]:
data

,FAMILY-ID,id,Start-Date,hospitaltype,T-duration,P-date,Referred,procedure,cost,claim-duration,claim-status,Day,label
0,1,1,2020-01-11,public,7,2020-01-12,NO,2,3000,7,SUCCESS,12,3
1,1,1,2020-01-11,public,7,2020-01-15,NO,6,4000,7,SUCCESS,15,3
2,1,1,2020-02-21,public,1,2020-01-22,NaN,5,0,4,PENDING,22,1
3,1,2,2020-02-24,private,2,2020-02-24,NO,4,6000,10,PENDING,24,2
4,1,2,2020-02-25,private,2,2020-02-26,NO,3,5000,3,SUCCESS,26,3
5,1,2,2020-02-28,public,1,2020-02-29,NO,1,3500,2,SUCCESS,29,2
6,1,3,2020-03-02,private,4,2020-03-03,NO,3,4000,4,PENDING,03,2
7,1,3,2020-03-04,private,3,2020-03-05,NO,2,3000,2,PENDING,05,2
8,1,3,2020-03-06,public,2,2020-03-07,NO,4,6000,5,SUCCESS,07,3
9,2,4,2020-03-08,public,3,2020-03-09,NO,1,6000,5,SUCCESS,09,3


In [ ]:
print(data.head())
data.shape

   FAMILY-ID  User id Start-Date  ...  cost  claim-duration claim-status
0          1        1 2020-01-11  ...  3000               7      SUCCESS
1          1        1 2020-01-11  ...  4000               7      SUCCESS
2          1        1 2020-02-21  ...     0               4      PENDING
3          1        2 2020-02-24  ...  6000              10      PENDING
4          1        2 2020-02-25  ...  5000               3      SUCCESS

[5 rows x 11 columns]


(36, 11)

In [ ]:
data.rename(columns = {"User id":"id"},inplace=True)
data.head()

,FAMILY-ID,id,Start-Date,hospitaltype,T-duration,P-date,Referred,procedure,cost,claim-duration,claim-status
0,1,1,2020-01-11,public,7,2020-01-12,NO,2,3000,7,SUCCESS
1,1,1,2020-01-11,public,7,2020-01-15,NO,6,4000,7,SUCCESS
2,1,1,2020-02-21,public,1,2020-01-22,NaN,5,0,4,PENDING
3,1,2,2020-02-24,private,2,2020-02-24,NO,4,6000,10,PENDING
4,1,2,2020-02-25,private,2,2020-02-26,NO,3,5000,3,SUCCESS


In [ ]:
import numpy as np
data["procedure"] = data["procedure"].astype("str")

In [ ]:
output = pd.DataFrame(columns=["id","procedure"])
for i,j in data.groupby(["id","Start-Date"]):
  col1 = j.iloc[0]["id"]
  col2 = j["procedure"].str.cat(sep = ",")
  output = output.append({"id":col1,"procedure":col2},ignore_index = True)
print(output.head())

  id procedure
0  1       2,6
1  1         5
2  2         4
3  2         3
4  2         1


In [ ]:
output["procedure"].head()

0    2,6
1      5
2      4
3      3
4      1
Name: procedure, dtype: object

In [ ]:
output.head()

,id,procedure
0,1,"2,6"
1,1,5
2,2,4
3,2,3
4,2,1


In [ ]:
data["P-date"] = data["P-date"].astype("str")
data["Day"] = data["P-date"].str.split("-").str[2]
data["Day"] = data["Day"].replace(np.nan,0)

In [ ]:
def value(a,b,c):
  sum= 0
  if(a > 1):
    sum += 1

  if(b == "SUCCESS"):
    sum += 1
  
  if(int(c) > 0):
    sum += 1

  return(sum)

In [ ]:
label = []
for index, row in data.iterrows():
 label.append(value(row["T-duration"],row["claim-status"],row["Day"]))

In [ ]:
data["label"] = label

In [ ]:
data.head()

,FAMILY-ID,id,Start-Date,hospitaltype,T-duration,P-date,Referred,procedure,cost,claim-duration,claim-status,Day,label
0,1,1,2020-01-11,public,7,2020-01-12,NO,2,3000,7,SUCCESS,12,3
1,1,1,2020-01-11,public,7,2020-01-15,NO,6,4000,7,SUCCESS,15,3
2,1,1,2020-02-21,public,1,2020-01-22,NaN,5,0,4,PENDING,22,1
3,1,2,2020-02-24,private,2,2020-02-24,NO,4,6000,10,PENDING,24,2
4,1,2,2020-02-25,private,2,2020-02-26,NO,3,5000,3,SUCCESS,26,3


In [ ]:
data.shape

(36, 13)

In [ ]:
means = []
for x,y in data.groupby(["id","Start-Date"]):
  means.append(y["label"].mean())

In [ ]:
means

In [ ]:
output["mean"] = means

In [ ]:
output.head()

,id,procedure,mean
0,1,"2,6",3.0
1,1,5,1.0
2,2,4,2.0
3,2,3,3.0
4,2,1,2.0


In [ ]:
def status(x):
  if(x >= 2):
    return("Satisfied")
  return ("Unsatisfied")

In [ ]:
output["claim"] = output["mean"].map(lambda x : status(x))

In [ ]:
output = output.drop("mean",axis = 1)
output

,id,procedure,claim
0,1,"2,6",Satisfied
1,1,5,Unsatisfied
2,2,4,Satisfied
3,2,3,Satisfied
4,2,1,Satisfied
5,3,3,Satisfied
6,3,2,Satisfied
7,3,4,Satisfied
8,4,"1,4",Satisfied
9,4,3,Unsatisfied


In [ ]:
output.to_excel("/content/drive/MyDrive/initial_output.xlsx",index = False)

In [ ]:
output = output.drop(["claim"],axis = 1)

In [ ]:
unique_val = data["procedure"].unique()
len(unique_val)

6

##Task 1

In [ ]:
def convert(x):
  return x.split(",")

In [ ]:
output["procedure"] = output["procedure"].map(lambda x : convert(x))

In [ ]:
output.head()

,id,procedure
0,1,"[P1, P3]"
1,1,[P6]
2,2,[P5]
3,2,[P4]
4,2,[P2]


##Task 2

In [ ]:
output1 = output.groupby("id")["procedure"].apply(tuple).reset_index()

In [ ]:
output1

,id,procedure
0,1,"([P1, P3], [P6])"
1,2,"([P5], [P4], [P2])"
2,3,"([P3], [P1], [P5])"
3,4,"([P5, P2], [P1])"
4,5,"([P4], [P6], [P3])"
5,6,"([P4], [P3], [P2])"
6,7,"([P4], [P6, P1])"
7,8,"([P3], [P6, P2])"
8,9,"([P4], [P2], [P1])"
9,10,"([P6, P5], [P4])"


In [ ]:
for key,value in output1.iterrows():
  print(value["procedure"])

In [ ]:
#Original Data
data = pd.DataFrame(output["procedure"])
patients = len(data)

In [ ]:
output.head()

,id,procedure
0,1,"[P1, P3]"
1,1,[P6]
2,2,[P5]
3,2,[P4]
4,2,[P2]


In [ ]:
s = output["procedure"][0]
s

['P1', 'P3']

##Task 3

In [ ]:
#Sequence Generation
final = []
count = 0
for p in range(patients):              #Rows
  s = (data["procedure"][p])
  s = tuple(s.split(","))
  for i in range(1,4):                  #Procedures          
    for t in range(len(s)-i+1):         #Columns
      final.append((s[t:t+i]))
      print(p,i,t)

In [ ]:
df = pd.DataFrame({'key': final,
                   'data': 1}, columns=['key', 'data'])

In [ ]:
data_count = df.groupby("key").sum().reset_index()

In [ ]:
data_count

,key,data
0,"(P1,)",4
1,"(P2,)",4
2,"(P3,)",5
3,"(P4,)",5
4,"(P5,)",3
5,"(P6,)",4


In [ ]:
#Removing the rows with sequence length more than 3
def filter(a):
  if len(a) > 3:
    return 1
  return 0

In [ ]:
data_count["new_data"] = data_count["key"].apply(filter)

In [ ]:

data_count = data_count.set_index("new_data")
#data_count = data_count.drop(1,axis = 0)

In [ ]:
data_count = data_count.reset_index()

In [ ]:
data_count = data_count.drop(["new_data"],axis = 1)

In [ ]:
data_count

,key,data
0,"(P1,)",5
1,"(P1, P3)",1
2,"(P2,)",5
3,"(P3,)",5
4,"(P4,)",6
5,"(P5,)",4
6,"(P5, P2)",1
7,"(P6,)",5
8,"(P6, P1)",1
9,"(P6, P2)",1


In [ ]:
unique_val

array(['P1', 'P3', 'P6', 'P5', 'P4', 'P2'], dtype=object)

In [ ]:
#Generating and adding the repeated sequences
repeat = []
for i in unique_val:
  for j in range(1,7):
    c = i.split() * j
    repeat.append(c)

In [ ]:
added = []
count = 0
for i in range(len(repeat)):
  count = 0
  for j in (data_count["key"]):
    if tuple(repeat[i]) != j:
      count += 1
  if (count == len(data_count) and len(tuple(repeat[i])) <= 3):   
    added.append(tuple(repeat[i]))

In [ ]:
len(added)
j = 0
counts = []
for i in range(len(added)):
  counts.append(j)

In [ ]:
added

new = pd.DataFrame({
    "key" : added,
    "data" : counts
})

In [ ]:
data_count1 = data_count.append(new,ignore_index=True)
data_count1

,key,data
0,"(P1,)",5
1,"(P1, P3)",1
2,"(P2,)",5
3,"(P3,)",5
4,"(P4,)",6
5,"(P5,)",4
6,"(P5, P2)",1
7,"(P6,)",5
8,"(P6, P1)",1
9,"(P6, P2)",1


In [ ]:
data_count

,key,data
0,"(P1,)",5
1,"(P1, P3)",1
2,"(P2,)",5
3,"(P3,)",5
4,"(P4,)",6
5,"(P5,)",4
6,"(P5, P2)",1
7,"(P6,)",5
8,"(P6, P1)",1
9,"(P6, P2)",1


In [ ]:
#Calculating the support for all the sequences
support = []
element = []
sequence = []
n = len(data_count)

for i in range(0,n):
  a = str(data_count["key"][i]).replace("(","").replace(")","").replace("'","").replace(",","")
  for j in range(0,n):
    b = str(data_count["key"][j]).replace("(","").replace(")","").replace("'","").replace(",","")
    if a in b:
      if b[b.index(a)+len(a): ] != "":
          element.append(a)
          sequence.append(b[b.index(a)+len(a)+1: ])
          d = data_count["data"][j]/n
          support.append(round(d,2))
print(sequence)

['P3', 'P2', 'P1', 'P2', 'P5']


In [ ]:
a = str(data_count["key"][0]).replace("(","").replace(")","").replace("'","").replace(",","")
b = str(data_count["key"][1]).replace("(","").replace(")","").replace("'","").replace(",","")

print(a)
print(b)
if b[b.index(a)+len(a): ] != "":
  print("Yes")
  print(b[b.index(a)+len(a)+1: ])

P1
P1 P3
Yes
P3


In [ ]:
#Calculation for element count
element_count = []
for i in range(len(element)):
  a = element[i]
  for j in range(0,n):
    b = str(data_count["key"][j]).replace("(","").replace(")","").replace("'","").replace(",","")
    if a == b:
      element_count.append(data_count["data"][j])

In [ ]:
#Calculating the confidence for all the sequences
confidence = []
n = len(data_count)

for i in range(len(sequence)):
  a = sequence[i]
  for j in range(0,n):
    b = str(data_count["key"][j]).replace("(","").replace(")","").replace("'","").replace(",","")
    if a == b:
      c = support[i]/data_count["data"][j]
      confidence.append(round(c,2))
      break

In [ ]:
#Calculation for Lift
n = len(data_count)
lift = []

for i in range(len(sequence)):
  a = sequence[i]
  for j in range(0,n):
    b = str(data_count["key"][j]).replace("(","").replace(")","").replace("'","").replace(",","")
    if a == b:
      num = support[i]/n
      denom = element_count[i]/ ((n * data_count["data"][j]) / n)
      fin = (num/denom)
      lift.append(round(fin,4))
      break

In [ ]:
print(len(sequence))
print(len(element))
print(len(confidence))
print(len(lift))

5
5
5
5


In [ ]:
final = pd.DataFrame({"element": element,"sequence": sequence,"support":support,"confidence":confidence,"lift":lift},
                     columns = ["element","sequence","support","confidence","lift"])

final

,element,sequence,support,confidence,lift
0,P1,P3,0.09,0.02,0.0082
1,P5,P2,0.09,0.02,0.0102
2,P6,P1,0.09,0.02,0.0082
3,P6,P2,0.09,0.02,0.0082
4,P6,P5,0.09,0.02,0.0065


In [ ]:
sum(final["lift"])

0.041299999999999996